In [5]:
import pickle
import torch
import numpy as np
import pandas as pd
import gpytorch
from sklearn.decomposition import PCA

## loading the data from ../train_generations.pkl
with open('../train_generations.pkl', 'rb') as f:
    data = pickle.load(f)
    print(type(data))
    data = pd.DataFrame(data)
 

<class 'dict'>


In [6]:
for key in sorted(data.keys()):
        for next_key in sorted(data[key].keys()):
            if next_key == "responses":
                response_data = (data[key][next_key])
                if not response_data:
                    continue
                first_response = response_data[0]
                print("AAAAAAAAAAAAAA")
                print(first_response)
                token_embeddings = first_response[2] 
                print("BBBBBBBBBBBBBBB")
                print(token_embeddings)
                print("AAAAAAAAAAAAAA")
                
                

AAAAAAAAAAAAAA
('Trumpets and trombones are most commonly pitched at Bb.', [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.08954261988401413, 0.0, -0.3505175709724426], tensor([[-2.8837,  6.8692,  1.1943,  ..., -1.6161, -0.4493, -2.8256]]), 0.0)
BBBBBBBBBBBBBBB
tensor([[-2.8837,  6.8692,  1.1943,  ..., -1.6161, -0.4493, -2.8256]])
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
("Every good boy deserves favour' is a mnemonic in?", [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], tensor([[-2.5785,  2.5525,  3.6656,  ..., -5.7184, -2.0659, -1.0848]]), 0.0)
BBBBBBBBBBBBBBB
tensor([[-2.5785,  2.5525,  3.6656,  ..., -5.7184, -2.0659, -1.0848]])
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
('John Lewis', [0.0, 0.0], tensor([[ 2.9588,  2.2058,  3.7353,  ..., -5.1753, -2.8140,  1.6127]]), 0.0)
BBBBBBBBBBBBBBB
tensor([[ 2.9588,  2.2058,  3.7353,  ..., -5.1753, -2.8140,  1.6127]])
AAAAAAAAAAAAAA
AAAAAAAAAAAAAA
('een Elizabeth II has always had corgis as pets. She has owned 30 corgis in her lifetime, according

In [7]:
# Let's assume df is already in memory
def extract_embeddings(df, limit=None):
    embeddings = []
    
    for key in sorted(data.keys()):
        for next_key in sorted(data[key].keys()):
            if next_key == "responses":
                try:
                    response_data = data[key][next_key]
                    response_data = data[key][next_key]
                    if not response_data:
                        continue
                    first_response = response_data[0]
                    token_embeddings = response_data[0][2]

                    if isinstance(token_embeddings, torch.Tensor) and token_embeddings.ndim == 2:
                        sentence_embedding = token_embeddings.mean(dim=0)
                        embeddings.append(sentence_embedding.detach().cpu())
                        
                    else:
                        continue
                except Exception as e:
                    print(f"Error processing {key}: {e}")
                    
                if limit and len(embeddings) >= limit:
                    break

    return torch.stack(embeddings)

In [8]:
X = extract_embeddings(data, limit=500)

In [9]:
class GPLVMModel(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = gpytorch.variational.VariationalStrategy(self, inducing_points, variational_distribution)
        super().__init__(variational_strategy)
        
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Placeholder to receive sentence embeddings
def run_gplvm(embeddings, latent_dim=2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    X = torch.tensor(embeddings, dtype=torch.float32).to(device)

    # Reduce dimensionality initially via PCA (for inducing point selection)
    pca = PCA(n_components=latent_dim)
    Z_init = torch.tensor(pca.fit_transform(embeddings), dtype=torch.float32).to(device)

    model = GPLVMModel(inducing_points=Z_init[:20])  # 20 inducing points
    likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
    model = model.to(device)

    model.train()
    likelihood.train()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=X.size(0))
    
    for i in range(300):
        optimizer.zero_grad()
        output = model(Z_init)
        loss = -mll(output, X)
        loss.backward()
        optimizer.step()
        if i % 50 == 0:
            print(f'Iter {i}: loss = {loss.item()}')

    return model, likelihood, Z_init


In [10]:
results = run_gplvm(X, latent_dim=2)

/var/folders/b3/hhpkbnjn3tsgdz2s3hjt3rdw0000gn/T/ipykernel_76913/2718846782.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(embeddings, dtype=torch.float32).to(device)


RuntimeError: Numpy is not available